In [28]:
import csv
import pandas as pd

import requests
from bs4 import BeautifulSoup, SoupStrainer
import re
import numpy as np
import json
import time
bussiness_licences = "business-licences-hackathon.csv"
file = open(bussiness_licences, "r")
reader = pd.read_csv(file, sep=';')

/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
reader

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,Street,City,Province,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom
0,14,2180123,14-160030,0,Golden Trim Enterprises Inc,Floorhaus,Issued,2013-12-05T19:08:53-05:00,2014-12-31,Wholesale Dealer w/ Anc. Retail,...,W 3RD AV,Vancouver,BC,CA,V5Y 1G1,Mount Pleasant,4.0,262.0,2019-07-21T16:49:13-04:00,"{""type"": ""Point"", ""coordinates"": [-123.1098925..."
1,14,2180373,14-160220,0,Tamton Networking Inc,Tam Ton Training,Issued,2014-02-26T16:08:59-05:00,2014-12-31,Instruction,...,W PENDER ST,Vancouver,BC,CA,V6B 6N9,Downtown,0.0,130.0,2019-07-21T16:49:13-04:00,"{""type"": ""Point"", ""coordinates"": [-123.1067518..."
2,14,2180374,14-160221,0,Jack W Chow Insurance Services Ltd,NaN,Issued,2013-12-06T11:57:32-05:00,2014-12-31,Office,...,E PENDER ST,Vancouver,BC,CA,V6A 1S9,Downtown,10.0,130.0,2019-07-21T16:49:13-04:00,"{""type"": ""Point"", ""coordinates"": [-123.1040791..."
3,14,2180382,14-160226,0,Time Education Inc,NaN,Cancelled,NaN,NaN,Educational,...,GRANVILLE ST,Vancouver,BC,CA,V6M 4J7,Kerrisdale,2.0,51.0,2019-07-21T16:49:13-04:00,"{""type"": ""Point"", ""coordinates"": [-123.1401383..."
4,13,2180556,13-449866,0,(Maiko Sato),Nankurunaisa Education Services,Issued,2013-12-17T16:18:45-05:00,2013-12-31,Referral Services,...,NaN,Vancouver,BC,CA,NaN,Riley Park,1.0,62.0,2019-07-21T16:49:13-04:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569022,21,3729711,21-147195,0,(Neil-Hung Tsao),K8 Strings,Pending,NaN,NaN,Repair/ Service/Maintenance,...,RENFREW ST,Vancouver,BC,CA,V5K 4C9,Hastings-Sunrise,1.0,155.0,2021-02-22T03:08:55-05:00,"{""type"": ""Point"", ""coordinates"": [-123.0446858..."
569023,21,3729717,21-147201,0,(Jeffrey Liu),Fraserview Bike Mechanic,Issued,2020-12-05T03:36:00-05:00,2021-12-31,Repair/ Service/Maintenance,...,NaN,Vancouver,BC,CA,NaN,Killarney,0.0,233.0,2021-02-22T03:08:55-05:00,NaN
569024,21,3729718,21-147202,0,24X7 Groceries Ltd,Cell Care Phone Repair,Issued,2020-12-31T19:59:16-05:00,2021-12-31,Repair/ Service/Maintenance,...,MAIN ST,Vancouver,BC,CA,V5T 3G8,Mount Pleasant,1.0,155.0,2021-02-22T03:08:55-05:00,"{""type"": ""Point"", ""coordinates"": [-123.1017488..."
569025,21,3729719,21-147203,0,Tickety-Boo Handyman Services Inc,NaN,Pending,NaN,NaN,Repair/ Service/Maintenance,...,NaN,Vancouver,BC,CA,NaN,Fairview,1.0,NaN,2021-02-22T03:08:55-05:00,NaN


In [26]:
# 35050 Business name is peoples names in brackets which are likely
# not to have results in canadabusinessregistries
df_no_nan = reader.dropna(subset=['BusinessName'])
businesses_names = df_no_nan[~df_no_nan.BusinessName.str.contains('(', regex=False)].BusinessName.unique()

# 35050 Business name is peoples names in brackets which are likely
# not to have results in canadabusinessregistries 
# so I am getting their business trade name instead. 
business_trade_names = df_no_nan[df_no_nan['BusinessName'].str.contains('(', regex=False)]["BusinessTradeName"].unique()

In [14]:
len(businesses_names)

58507

In [15]:
len(business_trade_names)

21606

In [7]:

# still not working properly. The list at the end does not contain all the entries in the content_json['docs']

# def get_business_info(businesses, current_business_list):
#     for business in businesses:
#         temp_dict = {}

#         temp_dict['Company_Name'] = business['Company_Name']

#         if 'BN' in business:
#             temp_dict['BN'] = business['BN']
#         else:
#             temp_dict['BN'] = "None"

#         temp_dict['Juri_ID'] = business['Juri_ID']
#         temp_dict['Status_State'] = business['Status_State']

#         if 'Status_Notes' in business:
#             temp_dict['Status_Notes'] = business['Status_Notes']
#         else: 
#             temp_dict['Status_Notes'] = "None"

#         if 'Reg_office_province' in business:
#             temp_dict['Reg_office_province'] = business['Reg_office_province']
#         else:
#             temp_dict['Reg_office_province'] = "None"

#         if 'Reg_office_city' in business:
#             temp_dict['Reg_office_city'] = business['Reg_office_city']
#         else:
#             temp_dict['Reg_office_city'] = "None"

#         if 'Display_Date' in business:
#             temp_dict['Display_Date'] = business['Display_Date']
#         else:
#             temp_dict['Display_Date'] = "None"
    
#     current_business_list[business['MRAS_ID']] = temp_dict

In [19]:
"https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7BKenny%20Pui%20Man%20Wong%20%26%20Li%20Bing%20Shu%7D&location=BC&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc"

'https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7BKenny%20Pui%20Man%20Wong%20%26%20Li%20Bing%20Shu%7D&location=BC&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc'

In [23]:
## The & are mapped to %26 in the API 
string = "Kenny Pui Man Wong Li Bing Shu"
string = string.replace("&", "%26")
string

'Kenny Pui Man Wong Li Bing Shu'

In [27]:
from collections import defaultdict
current_business_list = defaultdict(dict)

# Deal with multiword business names
for business in businesses_names:
    business_name_list = business.split()
    if len(business_name_list) > 1:
        company_name = ("%20".join(business_name_list)).replace("&", "%26")
        company_name += "%7D"
    else: 
        company_name = business
    API = "https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7B"+ company_name +"&location=BC&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc"
    
    # get info for Business name
    response = requests.get(API)
    content_json = response.json()
    print(business)
    print(content_json['paging'])
    #print(len(content_json['docs']))
    #for attr in content_json:
        #print(business)
        #print(attr)
    # later can be replaced by function call
    #get_business_info(content_json['docs'], current_business_list)
#     for business in content_json['docs']:
#         temp_dict = {}

#         temp_dict['Company_Name'] = business['Company_Name']

#         if 'BN' in business:
#             temp_dict['BN'] = business['BN']
#         else:
#             temp_dict['BN'] = "None"

#         temp_dict['Juri_ID'] = business['Juri_ID']
#         temp_dict['Status_State'] = business['Status_State']

#         if 'Status_Notes' in business:
#             temp_dict['Status_Notes'] = business['Status_Notes']
#         else: 
#             temp_dict['Status_Notes'] = "None"

#         if 'Reg_office_province' in business:
#             temp_dict['Reg_office_province'] = business['Reg_office_province']
#         else:
#             temp_dict['Reg_office_province'] = "None"

#         if 'Reg_office_city' in business:
#             temp_dict['Reg_office_city'] = business['Reg_office_city']
#         else:
#             temp_dict['Reg_office_city'] = "None"

#         if 'Display_Date' in business:
#             temp_dict['Display_Date'] = business['Display_Date']
#         else:
#             temp_dict['Display_Date'] = "None"

#         current_business_list[business['MRAS_ID']] = temp_dict
     

Golden Trim Enterprises Inc
{'previous': 'page=0', 'next': 'page=0'}
Tamton Networking Inc
{'previous': 'page=0', 'next': 'page=0'}
Jack W Chow Insurance Services Ltd
{'previous': 'page=0', 'next': 'page=0'}
Time Education Inc
{'previous': 'page=0', 'next': 'page=0'}
Sentinel Peak Holdings Ltd
{'previous': 'page=0', 'next': 'page=0'}
Gyoza Bar Ltd
{'previous': 'page=0', 'next': 'page=0'}
0384503 BC Ltd
{'previous': 'page=0', 'next': 'page=0'}
Elk Products of Canada Inc
{'previous': 'page=0', 'next': 'page=0'}
Jilaev's Jewellery Ltd
{'previous': 'page=0', 'next': 'page=0'}
Hastings Club Hospitality Corp
{'previous': 'page=0', 'next': 'page=0'}
530077 BC Ltd
{'previous': 'page=0', 'next': 'page=0'}
Venue Nightclub Ltd
{'previous': 'page=0', 'next': 'page=0'}
E-Leung Investments Ltd
{'previous': 'page=0', 'next': 'page=0'}
Wing Yan Cheung and Pui Kam Au
{'previous': 'page=0', 'next': 'page=0'}
FW Recyclers Ltd
{'previous': 'page=0', 'next': 'page=0'}
Kenny Pui Man Wong & Li Bing Shu
{'pre

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
current_business_list

In [5]:
for _, business in content_json['docs']:
    print(_)

TypeError: string indices must be integers

In [13]:
"https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7BAmazon%7D+Status_State:Active&location=ON&page=1&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc"

'https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7BAmazon%7D+Status_State:Active&location=ON&page=1&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc'